In [ ]:
from PIL import Image, ImageDraw
import pytesseract
import argparse
import cv2
import os
import ipywidgets as widgets
import numpy as np
from ipywidgets import Checkbox
import tesserocr
from tesserocr import PyTessBaseAPI, PSM
from timeit import default_timer as timer
from sliders import move_x, move_y, angle, move_alpha, move_beta, checkbox
#import matplotlib.pyplot as plt

#image_path = "time.jpg"
#img = cv2.imread(image_path, 1)
video_path = "time10.mp4"
video = cv2.VideoCapture(video_path, cv2.CAP_FFMPEG)
_, img = video.read()
video.release()

In [ ]:
def rotate_image(angle, img):
    image = img.copy()
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

In [ ]:
def draw_lines_on_image_and_show(move_x, move_y, angle):
    global img
    img_to_draw = rotate_image(angle, img)
    height,width,_ = img.shape
    
        #vertical line 1
    cv2.line(img_to_draw, (move_x[0], 0), (move_x[0], height), (255,0,0), 20)
        #vertical line 2
    cv2.line(img_to_draw, (move_x[1], 0), (move_x[1], height), (0,255,0), 20)
        #horizontal line 1
    cv2.line(img_to_draw, (0, move_y[0]), (width, move_y[0]), (0,0,255), 20)
        #horizontal line 1
    cv2.line(img_to_draw, (0, move_y[1]), (width, move_y[1]), (50,50,50), 20)
    
    image_bytes = cv2.imencode('.jpg', img_to_draw)[1].tobytes()
    image_widget = widgets.Image(value= image_bytes,
                                  format='jpg',
                                  width=600,
                                  height=400,
                                 )
    display(image_widget)
    
out = widgets.interactive_output(draw_lines_on_image_and_show, {'move_x': move_x, 'move_y': move_y, 'angle': angle})
widgets_img_rangeY = widgets.HBox([out, move_y])
slider_img = widgets.VBox([move_x, angle, widgets_img_rangeY])  
display(slider_img)

In [ ]:
img_to_draw = rotate_image(angle.value, img)

cropped_image = img_to_draw[move_y.value[1]:move_y.value[0], move_x.value[0]:move_x.value[1]]

gray = None

def show_cropped(move_alpha, move_beta, checkbox):
    global cropped_image, gray
    gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    gray = cv2.convertScaleAbs(gray, alpha=move_alpha, beta=move_beta)
    if checkbox == True:
        gray = np.invert(gray)
    cropped_bytes = cv2.imencode('.jpg', gray)[1].tobytes()
    ajusted_image = widgets.Image(
        value=cropped_bytes,
        format='jpg',
        )
    display(ajusted_image)
    
out3 = widgets.interactive_output(show_cropped, {'move_alpha': move_alpha, 'move_beta': move_beta, 'checkbox': checkbox})

widgets_sliders = widgets.VBox([move_alpha, move_beta, checkbox])
widgets_outputs = widgets.HBox([out3, widgets_sliders])
display(widgets_outputs)

In [ ]:
cropped_bytes = cv2.imencode('.jpg', gray)[1].tobytes()
ajusted_image = widgets.Image(
        value=cropped_bytes,
        format='jpg',
        )
display(ajusted_image)

In [ ]:
video_path = "zegar.mp4"
video = cv2.VideoCapture(video_path, cv2.CAP_FFMPEG)
#start = timer()
try:
    while True:
        read_return, captured_frame = video.read()
        if not read_return:
            break
        cropped_image = captured_frame[move_y.value[1]:move_y.value[0], move_x.value[0]:move_x.value[1]]
        im_pil = Image.fromarray(cropped_image)
        print(tesserocr.image_to_text(im_pil))  # print ocr text from image
        #undistorted = cv2.undistort(captured_frame, cameraMatrix, distCoeffs, None, cameraMatrix_rescaled)
        #image_bytes = cv2.imencode('.jpg', captured_frame)[1].tobytes()
        #display_handle.update(Image(data=image_bytes))
except KeyboardInterrupt:
    pass
finally:
    video.release()
#end = timer()
#print(end - start)

![alt text](paint.jpg "Title")

In [ ]:
filename = "{}.jpg".format(os.getpid())
cv2.imwrite(filename, gray)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(Image.open(filename))
os.remove(filename)
print(text)